In [35]:
from pyzotero import zotero
from getpass import getpass
import pandas as pd
from settings import LIBRARY_ID, LIBRARY_TYPE, ZOLTERO_KEY 
import ipdb
import os
import csv

In [29]:
## generate pdfs
file = "NI_43_101_Nickel_v0b.csv"
url = 'https://w3id.org/usgs/z/4530692/'

In [30]:
file_names = pd.read_csv(file)

In [31]:
id_names = file_names["id"]
downloads = []
for name in id_names:
    values = name.split("_")
    downloads.append(url+values[-1])

In [49]:
downloads[120:125] # need to download

['https://w3id.org/usgs/z/4530692/Z42DFTGR',
 'https://w3id.org/usgs/z/4530692/E6IDCBEU',
 'https://w3id.org/usgs/z/4530692/QN9PE72A',
 'https://w3id.org/usgs/z/4530692/QV3ZMRTC',
 'https://w3id.org/usgs/z/4530692/HBBC2BNM']

In [11]:
zot = zotero.Zotero(LIBRARY_ID, LIBRARY_TYPE, ZOLTERO_KEY)

if True:
    # ipdb.set_trace()
    attachments = zot.everything(zot.items(itemType='attachment'))

    def file_meta_from_item(item):
        if 'up' in item['links'] and 'enclosure' in item['links'] and item['links']['enclosure']['type'] == 'application/pdf':
            lib_id = item['library']['id']
            meta_key = item['links']['up']['href'].split('/')[-1]
            attachment_key = item['links']['self']['href'].split('/')[-1]
            linked_file_name = f"{lib_id}_{meta_key}_{attachment_key}.pdf"
            original_file_name = item['links']['enclosure']['title']

            return {
                'library_id': lib_id,
                'metadata_key': meta_key,
                'attachment_key': attachment_key,
                'linked_file_name': linked_file_name,
                'original_file_name': original_file_name
            }

    pdf_attachments = [file_meta_from_item(i) for i in attachments]
    print(f"Amount of attachements {pdf_attachments}")
    pdf_attachments = [i for i in pdf_attachments if i is not None]

    print("MAX VERSION:", max([i['version'] for i in attachments]))

    df_pdf_attachments = pd.DataFrame(pdf_attachments)
    print(df_pdf_attachments.head())
    df_pdf_attachments.to_parquet(f'{lib_id}.parquet')

    sample_record = df_pdf_attachments.sample().iloc[0].to_dict()
    print(sample_record['attachment_key'])
    print(sample_record['linked_file_name'])
    
     
    zot.dump(
        sample_record['attachment_key'],
        filename=sample_record['linked_file_name'],
        path='./'
    )


data = pd.read_parquet(f'{lib_id}.parquet')

for i, item in data.iterrows():
    print(i, item['linked_file_name'])
    zot.dump(
        item['attachment_key'],
        filename=item['linked_file_name'],
        path='./reports/nickel'
    )

KeyboardInterrupt: 

In [ ]:
path = "./reports/nickel"

for filename in os.listdir(path):
    print(filename)
    if filename not in id_names:
        os.remove(filename)